<a href="https://colab.research.google.com/github/Kian-2108/colab-notebooks/blob/main/MMM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
!pip install --upgrade google-meridian[colab,and-cuda] -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.6/615.6 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.3/363.3 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.9/24.9 MB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 895.7/895.7 kB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 577.2/577.2 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.5/192.5 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.3/130.3 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.6/217.6 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import arviz as az
import IPython
from meridian import constants
from meridian.analysis import analyzer
from meridian.analysis import formatter
from meridian.analysis import optimizer
from meridian.analysis import summarizer
from meridian.analysis import visualizer
from meridian.data import data_frame_input_data_builder
from meridian.data import test_utils
from meridian.model import model
from meridian.model import prior_distribution
from meridian.model import spec
import numpy as np
import pandas as pd
# check if GPU is available
from psutil import virtual_memory
import tensorflow as tf
import tensorflow_probability as tfp

ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))
print(
    'Num GPUs Available: ',
    len(tf.config.experimental.list_physical_devices('GPU')),
)
print(
    'Num CPUs Available: ',
    len(tf.config.experimental.list_physical_devices('CPU')),
)

Your runtime has 13.6 gigabytes of available RAM

Num GPUs Available:  1
Num CPUs Available:  1


In [ ]:
df = pd.read_excel("/content/Dow Jones - POC dataset - modified_poc_dataset.xlsx")
df

,week_start,asa_app,bing_display,bing_search,dv360_display,dv360_display_or_olv,facebook_social,google_display,google_display_or_olv,google_search,...,commissions_affiliate,placement_affiliate,brand_spend,redbox_app,liftoff_app,orders,news_anomaly,holiday_flag,sale_flag,wsj_emails_total
0,2022-01-03,19.78,92.44,34854.24,90.04,85.74,596157.82,71.58,73842.95,151262.48,...,55999.18,31.45,31486.48,54.76,85.77,16612,0,1,1,2110938.44
1,2022-01-10,68.50,26.03,27042.74,57.47,71.62,665526.63,56.17,77942.52,128322.68,...,44718.14,53.24,29486.99,90.34,67.79,18868,0,1,1,1525313.67
2,2022-01-17,24.45,59.16,35169.95,54.32,52.17,737263.23,29.59,47934.12,92450.86,...,54623.80,16.66,27281.34,60.28,66.14,14425,0,1,0,2015786.95
3,2022-01-24,41.46,20.58,30953.54,71.58,84.55,505900.98,94.87,42025.72,109841.72,...,62220.97,92.68,33625.09,27.79,21.02,8621,0,0,1,1742444.38
4,2022-01-31,78.64,26.52,27245.21,16.47,18.09,400583.55,59.54,60106.38,89608.01,...,39405.97,16.90,46937.51,19.39,67.63,11823,0,1,0,1698713.22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,2025-06-09,13.80,91.78,43724.33,220564.87,269361.63,4304.78,197077.89,163025.91,287413.35,...,13.16,52.58,8463.61,24.51,50.68,17023,0,1,1,1684932.27
180,2025-06-16,68.87,11.61,39367.43,117648.14,127862.20,1804.16,152803.49,124444.47,266532.15,...,57.94,69.60,32243.95,17.71,21.26,13387,0,0,0,1743136.52
181,2025-06-23,85.61,81.43,32398.98,76665.13,99145.28,38.75,98629.63,96447.10,132941.19,...,18.50,34.36,7681.45,48.01,96.25,7912,1,1,1,956960.97
182,2025-06-30,60.07,47.40,25832.44,72318.32,54702.05,62.94,99296.26,116936.85,165226.98,...,12.79,12.06,92.70,37.64,55.59,8294,1,1,0,706958.78


In [ ]:
kpi_type = "non_revenue"

default_kpi_column = "orders"

default_time_column = "week_start"
default_media_time_column = "week_start"

media_cols = [
    "asa_app",
    "bing_display",
    "bing_search",
    "dv360_display",
    "dv360_display_or_olv",
    "facebook_social",
    "google_display",
    "google_display_or_olv",
    "google_search",
    "linkedin_social",
    "meta_app",
    "reddit_social",
    "snapchat_social",
    "taptica_app",
    "twitter_social",
    "commissions_affiliate",
    "placement_affiliate",
    "brand_spend",
    "redbox_app",
    "liftoff_app",
    ]

media_spend_cols = [f"{col}_spend" for col in media_cols]

non_media_treatment_cols = [
    "sale_flag",
    "wsj_emails_total",
    ]

control_cols = [
    "news_anomaly",
    "holiday_flag",
]

weights = {col: 1 for col in media_cols}
normalized_weights = {col: weight/len(weights) for col, weight in weights.items()}

for col in media_cols:
  df[f"{col}_spend"] = df["brand_spend"]*normalized_weights[col]

In [ ]:
builder = data_frame_input_data_builder.DataFrameInputDataBuilder(
    kpi_type = kpi_type,
    default_kpi_column = default_kpi_column,
    default_time_column = default_time_column,
    default_media_time_column = default_media_time_column,
    )

In [ ]:
builder = (
    builder
    .with_kpi(
        df = df,
        kpi_col = default_kpi_column,
        time_col = default_time_column,
    )
    .with_media(
        df = df,
        media_cols = media_cols,
        media_spend_cols = media_spend_cols,
        media_channels = media_cols,
        )
    .with_non_media_treatments(
        df = df,
        non_media_treatment_cols = non_media_treatment_cols,
        )
    .with_controls(
        df = df,
        control_cols = control_cols,
        )
)

In [ ]:
data = builder.build()

/usr/local/lib/python3.12/dist-packages/meridian/data/input_data.py:471: UserWarning: Consider setting custom priors, as kpi_type was specified as `non_revenue` with no `revenue_per_kpi` being set. Otherwise, the total media contribution prior will be used with `p_mean=0.4` and `p_sd=0.2`. Further documentation available at https://developers.google.com/meridian/docs/advanced-modeling/unknown-revenue-kpi-custom#set-total-paid-media-contribution-prior
  warnings.warn(


In [ ]:
roi_mu = 0.2  # Mu for ROI prior for each media channel.
roi_sigma = 0.9  # Sigma for ROI prior for each media channel.
prior = prior_distribution.PriorDistribution(
    roi_m=tfp.distributions.LogNormal(roi_mu, roi_sigma, name=constants.ROI_M)
)
model_spec = spec.ModelSpec(prior=prior)

mmm = model.Meridian(input_data=data, model_spec=model_spec)

/usr/local/lib/python3.12/dist-packages/meridian/model/model.py:66: UserWarning: In a nationally aggregated model, the `media_effects_dist` will be reset to `normal`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/meridian/model/model.py:867: UserWarning: Consider setting custom ROI priors, as kpi_type was specified as `non_revenue` with no `revenue_per_kpi` being set. Otherwise, the total media contribution prior will be used with `p_mean=0.4` and `p_sd=0.2`. Further documentation available at  https://developers.google.com/meridian/docs/advanced-modeling/unknown-revenue-kpi-custom#set-total-paid-media-contribution-prior
  warnings.warn(


In [ ]:
%%time
mmm.sample_prior(500)
mmm.sample_posterior(
    n_chains=10, n_adapt=2000, n_burnin=500, n_keep=1000, seed=0
)

/usr/local/lib/python3.12/dist-packages/meridian/model/prior_distribution.py:915: UserWarning: Hierarchical distribution parameters must be deterministically zero for national models. tau_g_excl_baseline has been automatically set to Deterministic(0).
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/meridian/model/prior_distribution.py:915: UserWarning: Hierarchical distribution parameters must be deterministically zero for national models. eta_m has been automatically set to Deterministic(0).
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/meridian/model/prior_distribution.py:915: UserWarning: Hierarchical distribution parameters must be deterministically zero for national models. eta_rf has been automatically set to Deterministic(0).
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/meridian/model/prior_distribution.py:915: UserWarning: Hierarchical distribution parameters must be deterministically zero for national models. eta_om has been automatically set to

CPU times: user 7min 17s, sys: 15.1 s, total: 7min 32s
Wall time: 7min 37s


In [ ]:
model_diagnostics = visualizer.ModelDiagnostics(mmm)
model_diagnostics.plot_rhat_boxplot()

alt.LayerChart(...)

In [ ]:
model_fit = visualizer.ModelFit(mmm)
model_fit.plot_model_fit()

/usr/local/lib/python3.12/dist-packages/meridian/analysis/analyzer.py:590: UserWarning: The `aggregate_geos` argument is ignored in the national model. It will be reset to `True`.
  warnings.warn(


alt.LayerChart(...)

In [ ]:
mmm_summarizer = summarizer.Summarizer(mmm)

In [ ]:
model_diagnostics.predictive_accuracy_table()

/usr/local/lib/python3.12/dist-packages/meridian/analysis/analyzer.py:590: UserWarning: The `aggregate_geos` argument is ignored in the national model. It will be reset to `True`.
  warnings.warn(


,metric,geo_granularity,value
0,R_Squared,national,0.508717
1,MAPE,national,0.193112
2,wMAPE,national,0.184224


In [ ]:
media_summary = visualizer.MediaSummary(meridian=mmm)

In [ ]:
media_summary.plot_channel_contribution_area_chart()

/usr/local/lib/python3.12/dist-packages/meridian/analysis/analyzer.py:3103: UserWarning: Effectiveness is not reported because it does not have a clear interpretation by time period.
  warnings.warn(


alt.Chart(...)

In [ ]:
media_summary.plot_channel_contribution_bump_chart()

alt.Chart(...)

In [ ]:
media_summary.plot_contribution_waterfall_chart()

alt.LayerChart(...)

In [ ]:
media_summary.plot_contribution_pie_chart()

alt.LayerChart(...)

In [ ]:
media_summary.plot_spend_vs_contribution()

alt.FacetChart(...)

In [ ]:
media_summary.plot_roi_bar_chart()

alt.LayerChart(...)

In [ ]:
media_summary.plot_roi_bar_chart(include_ci=False)

alt.LayerChart(...)

In [ ]:
media_summary.plot_cpik()

alt.LayerChart(...)

In [ ]:
media_summary.plot_roi_vs_effectiveness()

alt.Chart(...)

In [ ]:
media_summary.plot_roi_vs_effectiveness(disable_size=True)

alt.Chart(...)

In [ ]:
media_summary.plot_roi_vs_mroi()

alt.Chart(...)

In [ ]:
media_summary.plot_roi_vs_mroi(
    selected_channels=media_cols,
    equal_axes=True
)

alt.Chart(...)

In [ ]:
media_effects = visualizer.MediaEffects(meridian=mmm)

In [ ]:
media_effects.plot_response_curves()

alt.FacetChart(...)

In [ ]:
media_effects.plot_response_curves(
    plot_separately=False,
    include_ci=False
)

alt.LayerChart(...)

In [ ]:
media_effects.plot_response_curves(
    plot_separately=False,
    num_channels_displayed=1
)

alt.LayerChart(...)

In [ ]:
media_effects.plot_adstock_decay()

alt.FacetChart(...)

In [ ]:
media_effects.plot_hill_curves()

{'media': alt.FacetChart(...)}

In [ ]:
# filepath = '/content/drive/MyDrive'
# start_date = '2021-01-25'
# end_date = '2024-01-15'
# mmm_summarizer.output_model_results_summary(
#     'summary_output.html', filepath, start_date, end_date
# )

# IPython.display.HTML(filename='/content/drive/MyDrive/summary_output.html')

In [ ]:
# %%time
# budget_optimizer = optimizer.BudgetOptimizer(mmm)
# optimization_results = budget_optimizer.optimize()

In [ ]:
# filepath = '/content/drive/MyDrive'
# optimization_results.output_optimization_summary(
#     'optimization_output.html', filepath
# )